In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
import itertools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import warnings

warnings.filterwarnings('ignore')

# sns.set_theme(style="whitegrid")

train_set = pd.read_csv('train_set.csv')
test_set = pd.read_csv('test_set.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Split the training dataset into features (x) and class (y)
x_train = train_set.copy()
x_train = train_set.drop("Class", axis=1)
y_train = train_set["Class"]

# Split the training dataset into features (x) and class (y)
x_test = test_set.copy()
x_test = x_test.drop("Class", axis=1)
y_test = test_set["Class"]

display(x_train.head())

train_len = len(y_train)
test_len = len(y_test)
total_len = train_len + test_len
print(total_len)
print(f"Number of training dataset records: {train_len}, {'{:.2f}%'.format(100 * train_len / total_len)} of total")
print(f"Number of test dataset records: {len(x_test)}, {'{:.2f}%'.format(100 * test_len / total_len)} of total")

,LowestVE/VCO2,PeakVO2Real,DiffPeakVO2,DiffPercentPeakVO2,MeanVO2,PeakVO2,MeanVCO2,O2PulseDiff,O2PulsePercent,VO2vsPeakVO2atVT
0,0.125760,-0.418146,0.363240,-0.858639,-0.015964,-0.418146,-0.312581,-0.702406,-0.574696,0.077734
1,0.638246,-1.135581,-0.584887,-0.521123,-1.149392,-1.135581,-1.196624,0.031928,-1.404812,-1.043617
2,-0.668593,-0.436081,-0.199208,-0.408618,-0.510841,-0.436081,-0.441169,-0.399094,-0.878007,-0.082459
3,-1.667941,0.066123,0.973898,1.005733,-0.287348,0.066123,-0.023258,0.654515,0.766261,0.382101
4,-0.540472,-0.238787,-1.050915,0.121763,0.335239,-0.238787,0.217844,-1.117464,-0.415058,0.542294


217
Number of training dataset records: 151, 69.59% of total
Number of test dataset records: 66, 30.41% of total


In [ ]:
print(x_train.columns, len(x_train.columns))

Index(['LowestVE/VCO2', 'PeakVO2Real', 'DiffPeakVO2', 'DiffPercentPeakVO2',
       'MeanVO2', 'PeakVO2', 'MeanVCO2', 'O2PulseDiff', 'O2PulsePercent',
       'VO2vsPeakVO2atVT'],
      dtype='object') 10


In [ ]:
# clf = LogisticRegression()
# clf.fit(x_train, y_train)

# y_pred = clf.predict(x_test)

In [ ]:
clf = LogisticRegression()
hyperparams = {
  "penalty": ["l1", "l2", "elasticnet", None],
  "C": [0.1, 1, 10, 100],
  "fit_intercept": [True, False],  # removed auto
  "solver": ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
  "max_iter": [100, 500],
}
grid_search = GridSearchCV(clf, hyperparams, cv=5)
_ = grid_search.fit(x_train, y_train)
print(grid_search.best_params_)

accuracies = []
precisions = []
recalls = []
f1s = []
for i in range(1000):
  y_pred = grid_search.predict(x_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  f1 = f1_score(y_test, y_pred, average='weighted')

  accuracies.append(accuracy)
  precisions.append(precision)
  recalls.append(recall)
  f1s.append(f1)

{'C': 0.1, 'fit_intercept': False, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}


In [ ]:
ave_accuracy = np.mean(accuracies)
ave_precision = np.mean(precisions)
ave_recall = np.mean(recalls)
ave_f1 = np.mean(f1s)

print("Average Scores:")
print("Accuracy:", ave_accuracy)
print("Precision:", ave_precision)
print("Recall:", ave_recall)
print("F1 Score:", ave_f1)

Average Scores:
Accuracy: 0.5606060606060606
Precision: 0.7259358288770054
Recall: 0.5606060606060606
F1 Score: 0.5013363632131959


In [ ]:
# matrix = confusion_matrix(y_test, y_pred)

# class_names = ['CardiacLim','Healthy','MuscleSkeletalLim','PulmonaryLim']
   
# plt.clf()

# # place labels at the top
# plt.gca().xaxis.tick_top()
# plt.gca().xaxis.set_label_position('top')

# # plot the matrix per se
# plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)

# # plot colorbar to the right
# plt.colorbar()
    
# fmt = 'd'

# # write the number of predictions in each bucket
# thresh = matrix.max() / 2.
# for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):

#     # if background is dark, use a white number, and vice-versa
#     plt.text(j, i, format(matrix[i, j], fmt),
#          horizontalalignment="center",
#          color="white" if matrix[i, j] > thresh else "black")
    
# tick_marks = np.arange(len(class_names))
# plt.xticks(tick_marks, class_names, rotation=45)
# plt.yticks(tick_marks, class_names)
# plt.tight_layout()
# plt.ylabel('True label', size=14)
# plt.xlabel('Predicted label', size=14)
# plt.show()